In [9]:
from sklearn.cross_decomposition import CCA

In [10]:
cca = CCA(n_components=2)

In [1]:
import pandas as pd 

X = pd.read_csv('Wild-type.csv', index_col=0)[:13509]
Y = pd.read_csv('DUSP6-KO.csv', index_col=0)[:13509]

In [2]:
import pandas as pd 


Y = pd.read_csv('test1_scaled.csv').to_numpy()
X = pd.read_csv('test2_scaled.csv').to_numpy()

In [3]:
Y = Y.T

In [4]:
X = X.T

In [ ]:
X

In [24]:
import tensorflow as tf
import tensorflow.keras.backend as T
from tensorflow.keras.layers import Layer

class DCCA(Layer):
    '''CCA layer is used compute the CCA objective

    # Input shape
        Arbitrary. Use the keyword argument `input_shape`
        (tuple of integers, does not include the samples axis)
        when using this layer as the first layer in a model.

    # Output shape
        Same shape as the input.

    # Arguments
        output_dim: output dimension, default 1, i.e., correlation coefficient
        use_all_singular_value: if use the top-k singular values
        cca_space_dim: the number of singular values, i.e., k

    '''

    def __init__(self, output_dim=1, use_all_singular_values=True, cca_space_dim=10, **kwargs):
        self.output_dim = output_dim
        self.cca_space_dim = cca_space_dim
        self.use_all_singular_values = use_all_singular_values
        super(DCCA, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        super(DCCA, self).build(input_shape)

    def call(self, x):
        r1 = tf.constant([1e-4])
        r2 = tf.constant([1e-4])
        eps = tf.constant([1e-12])
        o1 = o2 = tf.shape(x)[1] // 2

        H1 = T.transpose(x[:, 0:o1])
        H2 = T.transpose(x[:, o1:o1 + o2])

        one = tf.constant([1.0])
        m = tf.shape(H1)[1]
        m_float = tf.cast(m, 'float')

        # minus the mean value
        partition = tf.divide(one, m_float)
        H1bar = H1 - partition * tf.matmul(H1, tf.ones([m, m]))
        H2bar = H2 - partition * tf.matmul(H2, tf.ones([m, m]))

        # calculate the auto-covariance and cross-covariance
        partition2 = tf.divide(one, (m_float - 1))
        SigmaHat12 = partition2 * tf.matmul(H1bar, tf.transpose(H2bar))
        SigmaHat11 = partition2 * tf.matmul(H1bar, tf.transpose(H1bar)) + r1 * tf.eye(o1)
        SigmaHat22 = partition2 * tf.matmul(H2bar, tf.transpose(H2bar)) + r2 * tf.eye(o2)

        # calculate the root inverse of covariance matrices by using eigen decomposition
        D1, V1 = tf.linalg.eigh(SigmaHat11)
        D2, V2 = tf.linalg.eigh(SigmaHat22)

        # for stability
        D1_indices = tf.where(D1 > eps)
        D1_indices = tf.squeeze(D1_indices)
        V1 = tf.gather(V1, D1_indices)
        D1 = tf.gather(D1, D1_indices)

        D2_indices = tf.where(D2 > eps)
        D2_indices = tf.squeeze(D2_indices)
        V2 = tf.gather(V2, D2_indices)
        D2 = tf.gather(D2, D2_indices)

        pow_value = tf.constant([-0.5])
        SigmaHat11RootInv = tf.matmul(tf.matmul(V1, tf.linalg.diag(tf.pow(D1, pow_value))), tf.transpose(V1))
        SigmaHat22RootInv = tf.matmul(tf.matmul(V2, tf.linalg.diag(tf.pow(D2, pow_value))), tf.transpose(V2))

        Tval = tf.matmul(tf.matmul(SigmaHat11RootInv, SigmaHat12), SigmaHat22RootInv)

        if self.use_all_singular_values:
            # all singular values are used to calculate the correlation
            corr = tf.linalg.trace(T.sqrt(tf.matmul(tf.transpose(Tval), Tval)))
        else:
            # just the top outdim_size singular values are used
            TT = tf.matmul(tf.transpose(Tval), Tval)
            U, V = tf.raw_ops.SelfAdjointEigV2(input=TT)
            U_sort, _ = tf.nn.top_k(U, self.cca_space_dim)
            corr = T.sum(T.sqrt(U_sort))

        corr = tf.fill([tf.shape(x)[0], self.output_dim], corr)

        return -corr

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

    def get_config(self):
        config = {
            'output_dim': self.output_dim,
            'cca_dim': self.cca_dim,
            'use_all_singular_values': self.use_all_singular_values,
        }
        base_config = super(CCA, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [29]:
import tensorflow as tf
from tensorflow.keras.layers import Layer

class LogNormalizeLayer(Layer):
    def __init__(self, scale_factor=10000):
        super(LogNormalizeLayer, self).__init__()
        self.scale_factor = scale_factor

    def call(self, inputs):
        # Obliczanie sumy wartości dla każdej komórki
        total_counts = tf.reduce_sum(inputs, axis=-1, keepdims=True)

        # Normalizacja danych przez podzielenie przez sumę
        normalized_data = tf.divide(inputs, total_counts)

        # Przekształcenie logarytmiczne danych z uwzględnieniem skali i dodaniem 1
        log_transformed_data = tf.math.log(self.scale_factor * normalized_data + 1)

        return log_transformed_data


In [25]:
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, concatenate, Dropout
import tensorflow.keras.backend as K
from sklearn.model_selection import train_test_split

def constant_loss(y_true, y_pred):
    return y_pred

def mean_pred(y_true, y_pred):
    return K.mean(y_pred)


#train_set_x1, valid_set_x1, train_set_x2, valid_set_x2 = train_test_split(X, Y, test_size=0.20)

# size of the input for view 1 and view 2
input_shape1 = 2000
input_shape2 = 2000

# network settings
epoch_num = 100


batch_size = 2000
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import BatchNormalization

#load data
from keras.optimizers import Adam, RMSprop

input1 = Input(shape=(input_shape1, ), name='input1')
input2 = Input(shape=(input_shape2, ), name='input2')

expert_index = 0
# Definicja funkcji aktywacji
activation_model = LeakyReLU(alpha=0.2)

# Warstwy wejściowe
input1 = Input(shape=(input_shape1,))
input2 = Input(shape=(input_shape2,))

# Warstwy gęste dla widoku 1
normalizaction_1 = BatchNormalization()(input1)
dense1_1 = Dense(1024, activation=activation_model, name='view_1_1')(normalizaction_1)
dense1_2 = Dense(512, activation=activation_model, name='view_1_2')(dense1_1)
dense1_3 = Dense(256, activation=activation_model, name='view_1_3')(dense1_2)
output1 = Dense(25, activation='linear', name='view_1_4')(dense1_3)

# Warstwy gęste dla widoku 2
normalizaction_2 = BatchNormalization()(input1)
dense2_1 = Dense(1024, activation=activation_model, name='view_2_1')(normalizaction_2)
dense2_2 = Dense(512, activation=activation_model, name='view_2_2')(dense2_1)
dense2_3 = Dense(256, activation=activation_model, name='view_2_3')(dense2_2)
output2 = Dense(25, activation='linear', name='view_2_4')(dense2_3)

# Warstwa łącząca
shared_layer = concatenate([output1, output2], name='shared_layer')

# Normalizacja danych
shared_layer = BatchNormalization()(shared_layer)

cca_layer = DCCA(1, name='cca_layer')(shared_layer)

model = Model(inputs=[input1, input2], outputs=cca_layer)
model.compile(optimizer=RMSprop(lr=0.0001), loss=constant_loss, metrics=[mean_pred])
model.fit([X, Y], np.zeros(len(X)),
          batch_size=batch_size, epochs=epoch_num, shuffle=True, verbose=1)


Epoch 1/100
5/5 [==============================] - 3s 29ms/step - loss: -4.5911 - mean_pred: -4.5911
Epoch 2/100
5/5 [==============================] - 0s 30ms/step - loss: -9.7784 - mean_pred: -9.7784
Epoch 3/100
5/5 [==============================] - 0s 30ms/step - loss: -12.8094 - mean_pred: -12.8094
Epoch 4/100
5/5 [==============================] - 0s 30ms/step - loss: -14.9918 - mean_pred: -14.9918
Epoch 5/100
5/5 [==============================] - 0s 29ms/step - loss: -16.7082 - mean_pred: -16.7082
Epoch 6/100
5/5 [==============================] - 0s 29ms/step - loss: -18.1183 - mean_pred: -18.1183
Epoch 7/100
5/5 [==============================] - 0s 30ms/step - loss: -19.2857 - mean_pred: -19.2857
Epoch 8/100
5/5 [==============================] - 0s 31ms/step - loss: -20.2811 - mean_pred: -20.2811
Epoch 9/100
5/5 [==============================] - 0s 29ms/step - loss: -21.0997 - mean_pred: -21.0997
Epoch 10/100
5/5 [==============================] - 0s 29ms/step - loss: -21.

In [63]:
# evaluation for view_1
current_dcca = Model(model.input, model.get_layer(name='shared_layer').output)


pred_out = current_dcca.predict([X, Y])


423/423 [==============================] - 1s 3ms/step


In [35]:
df = pd.DataFrame(pred_out)

In [27]:

import numpy

import numpy
def total_correlation(X1, X2, k):
    r1 = 1e-4
    r2 = 1e-4

    n1 = X1.shape[1]
    n2 = X2.shape[1]
    m = X1.shape[0] #number of rows

    mean1 = numpy.mean(X1, axis=0)
    mean2 = numpy.mean(X2, axis=0)

    H1bar = X1 - numpy.tile(mean1, (m, 1))
    H2bar = X2 - numpy.tile(mean2, (m, 1))


    SigmaHat12 = (1.0 / (m - 1)) * numpy.dot(H1bar.T, H2bar) # cross-covariance matrix
    SigmaHat11 = (1.0 / (m - 1)) * numpy.dot(H1bar.T, H1bar) + r1 * numpy.identity(n1) # covariances
    SigmaHat22 = (1.0 / (m - 1)) * numpy.dot(H2bar.T, H2bar) + r2 * numpy.identity(n2) # covariances


    [D1, V1] = numpy.linalg.eigh(SigmaHat11) #Eigendecomposition for easy inversion
    [D2, V2] = numpy.linalg.eigh(SigmaHat22) #Eigendecomposition for easy inversion
    SigmaHat11RootInv = numpy.dot(numpy.dot(V1, numpy.diag(D1 ** -0.5)), V1.T) #
    SigmaHat22RootInv = numpy.dot(numpy.dot(V2, numpy.diag(D2 ** -0.5)), V2.T)
    T = numpy.dot(numpy.dot(SigmaHat11RootInv, SigmaHat12), SigmaHat22RootInv)


    [U, D, V] = numpy.linalg.svd(T)
    V = V.T
    Astar = numpy.dot(SigmaHat11RootInv, U[:, 0:k])
    Bstar = numpy.dot(SigmaHat22RootInv, V[:, 0:k])
    D = D[0:k]

    top_k_singular_values = D[:k]

    total_corr = numpy.sum(D)

    return Astar, total_corr, top_k_singular_values, Bstar


In [29]:
U, total_corr, singular_values, V = total_correlation(X, Y, k=25)
total_corr

19.65225721257287